In [1]:
"""
In this example, we'll provide some examples to guide the agent to answer how we'd like.
In particular, we'll convey that it should assume ETH network, set that network, and then
perform an ERC20 lookup.
"""

import json
import os

from emp_agents.agents.persistentagent import PersistentAgent, PersistentAgentConfig
from emp_agents.models.protocol.registry import ToolRegistry
from emp_agents.models.shared import (
    Example,
    UserMessage,
    AssistantMessage,
    ToolMessage,
    ToolCall,
    Message,
)
from emp_agents.providers import OpenAIProvider, OpenAIModelType

example: Example = (
    UserMessage(
        content="Tell me about 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2."
    ),
    [
        (
            AssistantMessage(
                content="We can use the ERC20Skill to get information about the token. But first, we need to set the network to Ethereum.",
                tool_calls=[
                    ToolCall(
                        id="1",
                        type="function",
                        function=ToolCall.Function(
                            name="set_network",
                            arguments=json.dumps({"network": "ethereum"})
                        ),
                    )
                ]
            ),
            ToolMessage(
                content="Network set to Ethereum.",
                tool_call_id="1",
            ),
        ),
        (
            AssistantMessage(
                content="With the network set we can now perform the ERC20 lookup.",
                tool_calls=[
                    ToolCall(
                        id="2",
                        type="function",
                        function=ToolCall.Function(
                            name="get_token_info",
                            arguments=json.dumps(
                                {"token_address": "0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2"}
                            ),
                        ),
                    )
                ],
            ),
            ToolMessage(
                content="The token at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2 is called WETH.",
                tool_call_id="2",
            ),
        ),
    ],
    AssistantMessage(
        content="WETH is a token on the Ethereum network.",
    ),
)

erc20_skill = ToolRegistry.get_skill("ERC20Skill")
network_skill = ToolRegistry.get_skill("NetworkSkill")

agent = PersistentAgent.from_config(
    PersistentAgentConfig(
        agent_id="dynamic_agent",
        name="Token Researcher",
        description="Tools for interacting with the blockchain",
        tools=[*erc20_skill, *network_skill],
        default_model=OpenAIModelType.gpt4o_mini,
        extra={
            "openai_api_key": os.environ.get("OPENAI_API_KEY"),
        },
    ),
    provider=OpenAIProvider(),
)

# Flatten example into messages we can feed into the agent.
example_messages: list[Message] = [
    example[0],
    *[msg for tpl in example[1] for msg in tpl],
    example[2],
]

agent.add_messages(example_messages)

print(agent.conversation_history)

info = await agent.answer(
    "Tell me about the ERC20 coin at 0x2260fac5e5542a773aa44fbcfedf7c193bc2c599."
)

[system: You are a helpful assistant, user: Tell me about 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2., assistant: We can use the ERC20Skill to get information about the token. But first, we need to set the network to Ethereum., tool: Network set to Ethereum., assistant: With the network set we can now perform the ERC20 lookup., tool: The token at 0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2 is called WETH., assistant: WETH is a token on the Ethereum network.]
[assistant: [<ToolCall function=name='get_token_info' arguments={'token_address': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'}>]]
[assistant: [<ToolCall function=name='set_network' arguments={'network': 'ethereum'}>]]
[assistant: [<ToolCall function=name='get_token_info' arguments={'token_address': '0x2260fac5e5542a773aa44fbcfedf7c193bc2c599'}>]]
[assistant: [<ToolCall function=name='set_network' arguments={'network': 'ethereum'}>]]
[assistant: [<ToolCall function=name='get_token_info' arguments={'token_address': '0x2260fac5e5542a7

CancelledError: 